`Import the modules`

In [271]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score

`Read the data from the csv file and make a pandas dataframe of that`

In [272]:
df = pd.read_csv('profiles.csv')

`Get some info about the dataframe`

In [ ]:
df.info()

In [ ]:
print('Some descriptive statistic data of numerical variables')
df.describe()

In [ ]:
print('You can see above that the height and the income columns include data unrealistic.')

In [ ]:
print('Unique values of income')
df.income.unique()

In [ ]:
df[df.income == -1].head(2)

In [ ]:
df.sex.replace(['m', 'f'], ['male', 'female'], inplace=True)

In [ ]:
print('The number of observations with income value -1 is: ', len(df[df.income == -1]))

In [ ]:
print('''We could drop the whole income column because -1 is not a usual categorical variable and definitely not
a continuous numerical variable. Thus we cannot use them or replace the values without avoiding bias.''')

In [ ]:
df = df.drop('income', axis=1)

In [ ]:
print('Unique values of height')
df.height.unique()

In [ ]:
print('''The smallest 18 year-old-man in the world is 29 inches, thus we can replace, or drop the values if they are lower than that.
https://www.mirror.co.uk/news/world-news/worlds-shortest-teenager-named-18-27050545''')

In [ ]:
df = df[~((df.height <= 29) | (df.height.isna()))]

In [ ]:
df.describe()

In [ ]:
df.height.unique()

`Make cms from inches`

In [ ]:
df.height = df.height*2.54

`Take a look at the "nan" values of the whole dataframe`

In [ ]:
df_isna = df.isna().sum().sort_values()
#df_isna

`Choose some and categorical and numerical variables and put them into a list`

In [ ]:
cats_and_nums = ['age', 'sex', 'orientation', 'height', 'status', 'drinks', 'body_type', 'smokes']

In [ ]:
for feature in cats_and_nums:
    print(df[feature].value_counts())

In [ ]:
def unique(features, data):
    for feature in features:
        if data[feature].dtype not in ['int64', 'float64']:
            data[feature] = data[feature].fillna('unknown')
            yield feature, data[feature].unique()

In [ ]:
test_1 = list(unique(cats_and_nums, df))

In [ ]:
test_1

`Making pie charts, countplots and histograms of the features from the list`

In [ ]:
def charts(features, dataframe):
    for feature in features:
        plt.figure(figsize=[10, 7])
        data = dataframe[feature]
        title = f'Distribution of {feature} of {len(data)} participants'
        if data.dtype == 'object':
            if data.nunique() < 6:
                plt.pie(data.value_counts(), autopct='%d%%', pctdistance=.85, explode=[0.05 for i in range(data.nunique())], colors=sns.color_palette('Set1'))
                plt.title(title)
                plt.legend(data.unique())
                plt.gcf().gca().add_artist(plt.Circle((0, 0), 0.7, facecolor='lightblue'))
            else:
                sns.countplot(x=data, saturation=0.75, palette='Set1')
                plt.title(title)
                plt.xticks(rotation=15, fontsize=9)
        elif data.dtype in ['int64', 'float64']:
            sns.histplot(x=feature, data = dataframe, hue='sex', bins=30, color=['darkred', 'darkblue'])
            plt.title(title)
            mean, median = np.mean(data).round(), np.median(data).round()
            plt.vlines([mean, median], colors=['red', 'blue'], ymin=0, ymax=11000)
            plt.text(x=mean*1.1, y=8000, s=f'Mean: {mean}')
            plt.text(x=median*.8, y=8000, s='Median')
        plt.show()
        plt.clf()

In [ ]:
test_2 = charts(cats_and_nums, df)

`Change the dtypes of "sex, orientation and status" categorical columns from object to int`

In [ ]:
df.sex = np.where(df['sex'] == 'male', 1, 0)
df.orientation = df.orientation.replace(['straight', 'bisexual', 'gay'], [2, 1, 0])
df['status'] = df['status'].replace(['single', 'available', 'seeing someone'], 1)
df['status'] = df['status'].replace(['married', 'unknown'], 0)

In [ ]:
encodeable = cats_and_nums[-3:]

In [ ]:
encodeable

In [ ]:
for uncoded_feature in encodeable:
    encoder = LabelEncoder()
    df[uncoded_feature] = encoder.fit_transform(df[uncoded_feature])
    

In [ ]:
df_model = df[cats_and_nums]

In [ ]:
df_model

`Make the predictor and the outcome variables.`

`X is equal to a pandas dataframe with the chosen features and y is equal to a pandas series with status`

In [ ]:
X = df[cats_and_nums[:-4]]
y = df.status

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=15)

In [ ]:
model1 = LogisticRegression()

In [ ]:
model1.fit(X_train, y_train)

In [ ]:
y_pred = model1.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred), model1.score(X_test, y_test)

In [ ]:
#df.sign.unique()

In [ ]:
sign_dict = {'gemini', 'cancer', 'pisces but it doesn&rsquo;t matter', 'pisces',
       'aquarius', 'taurus', 'virgo', 'sagittarius',
       'gemini but it doesn&rsquo;t matter',
       'cancer but it doesn&rsquo;t matter',
       'leo but it doesn&rsquo;t matter', nan,
       'aquarius but it doesn&rsquo;t matter',
       'aries and it&rsquo;s fun to think about',
       'libra but it doesn&rsquo;t matter',
       'pisces and it&rsquo;s fun to think about', 'libra',
       'taurus but it doesn&rsquo;t matter',
       'sagittarius but it doesn&rsquo;t matter',
       'scorpio and it matters a lot',
       'gemini and it&rsquo;s fun to think about',
       'leo and it&rsquo;s fun to think about',
       'cancer and it&rsquo;s fun to think about',
       'libra and it&rsquo;s fun to think about',
       'aquarius and it&rsquo;s fun to think about',
       'virgo but it doesn&rsquo;t matter',
       'scorpio and it&rsquo;s fun to think about',
       'capricorn but it doesn&rsquo;t matter', 'scorpio',
       'capricorn and it&rsquo;s fun to think about', 'leo',
       'aries but it doesn&rsquo;t matter', 'aries',
       'scorpio but it doesn&rsquo;t matter',
       'sagittarius and it&rsquo;s fun to think about',
       'libra and it matters a lot',
       'taurus and it&rsquo;s fun to think about',
       'leo and it matters a lot',
       'virgo and it&rsquo;s fun to think about',
       'cancer and it matters a lot', 'capricorn',
       'pisces and it matters a lot', 'aries and it matters a lot',
       'capricorn and it matters a lot', 'aquarius and it matters a lot',
       'sagittarius and it matters a lot', 'gemini and it matters a lot',
       'taurus and it matters a lot', 'virgo and it matters a lot'}

In [ ]:
df.location.unique()

In [ ]:
df.sex

In [ ]:
df.columns

In [ ]:
df_new = df.drop(['essay0','essay1', 'essay2', 'essay3', 'essay4', 'essay5', 'essay6', 'essay7', 'essay8', 'essay9',], axis=1)

In [ ]:
df_new.head(2)

In [ ]:
df_new.body_type.unique(), df_new.drugs.unique(), df_new.drinks.unique()

In [ ]:
features = list(df_new.columns)

In [ ]:
features

In [ ]:
df_new.ethnicity.unique()

In [ ]:
features.pop(6)

In [ ]:
np.max(df_new.last_online.unique())

In [ ]:
np.min(df_new.last_online.unique())

In [ ]:
fix_features = ['age', 'height', 'income', 'sex']

In [ ]:
varied_features = np.random.choice([feature for feature in features if feature != fix_features], 3)

In [ ]:
final_features = fix_features + list(varied_features)

In [ ]:
final_features

In [ ]:
pd.get_dummies(df_new[final_features], drop_first=True)

In [ ]:
df.status.unique()

In [ ]:
df.status.unique()

In [ ]:
y = df.status